# workforce.py
- Solve a classic model to assign workers to shifts  为工人安排值班的经典问题 
- This model is infeasible, so we do some tests to determine why 因为模型不可行, 需要做一些特别处理

## List data
  0. Shifts  轮班
  0. Workers  工人 

In [1]:
from gurobipy import *

m = Model("workforce")

# Number of workers required for each shift  14天2周轮班
shifts = [ "Mon1",  "Tue2", "Wed3",  "Thu4",  "Fri5",  "Sat6",  "Sun7",
           "Mon8",  "Tue9", "Wed10", "Thu11", "Fri12", "Sat13", "Sun14" ]

# 7个工人
workers = [ "Amy", "Bob", "Cathy", "Dan", "Ed", "Fred", "Gu" ]

## Indexed data
  0. Shift requirements: indexed by shifts
  0. Worker pay: indexed by workers

In [2]:
# 每天排班量 
shiftRequirements = {
"Mon1":  3, "Tue2":  2, "Wed3":  4, "Thu4":  4, "Fri5":  5, "Sat6":  6, "Sun7":  5,
"Mon8":  2, "Tue9":  2, "Wed10": 3, "Thu11": 4, "Fri12": 6, "Sat13": 7, "Sun14": 5 } 

# Amount each worker is paid to work one shift 每班所付报酬
pay = {
  "Amy":   10,
  "Bob":   12,
  "Cathy": 10,
  "Dan":   8,
  "Ed":    8,
  "Fred":  9,
  "Gu":    11 }

## Worker availability
We create a list of shifts when workers are available 工人的排班可用性
- Much more efficient to list only shifts that are possible
- Don't optimize shifts for workers who are unavailable!

In [3]:
# Worker availability
availability = tuplelist([
('Amy', 'Tue2'), ('Amy', 'Wed3'), ('Amy', 'Fri5'), ('Amy', 'Sun7'),
('Amy', 'Tue9'), ('Amy', 'Wed10'), ('Amy', 'Thu11'), ('Amy', 'Fri12'),
('Amy', 'Sat13'), ('Amy', 'Sun14'), ('Bob', 'Mon1'), ('Bob', 'Tue2'),
('Bob', 'Fri5'), ('Bob', 'Sat6'), ('Bob', 'Mon8'), ('Bob', 'Thu11'),
('Bob', 'Sat13'), ('Cathy', 'Wed3'), ('Cathy', 'Thu4'), ('Cathy', 'Fri5'),
('Cathy', 'Sun7'), ('Cathy', 'Mon8'), ('Cathy', 'Tue9'), ('Cathy', 'Wed10'),
('Cathy', 'Thu11'), ('Cathy', 'Fri12'), ('Cathy', 'Sat13'),
('Cathy', 'Sun14'), ('Dan', 'Tue2'), ('Dan', 'Wed3'), ('Dan', 'Fri5'),
('Dan', 'Sat6'), ('Dan', 'Mon8'), ('Dan', 'Tue9'), ('Dan', 'Wed10'),
('Dan', 'Thu11'), ('Dan', 'Fri12'), ('Dan', 'Sat13'), ('Dan', 'Sun14'),
('Ed', 'Mon1'), ('Ed', 'Tue2'), ('Ed', 'Wed3'), ('Ed', 'Thu4'),
('Ed', 'Fri5'), ('Ed', 'Sun7'), ('Ed', 'Mon8'), ('Ed', 'Tue9'),
('Ed', 'Thu11'), ('Ed', 'Sat13'), ('Ed', 'Sun14'), ('Fred', 'Mon1'),
('Fred', 'Tue2'), ('Fred', 'Wed3'), ('Fred', 'Sat6'), ('Fred', 'Mon8'),
('Fred', 'Tue9'), ('Fred', 'Fri12'), ('Fred', 'Sat13'), ('Fred', 'Sun14'),
('Gu', 'Mon1'), ('Gu', 'Tue2'), ('Gu', 'Wed3'), ('Gu', 'Fri5'),
('Gu', 'Sat6'), ('Gu', 'Sun7'), ('Gu', 'Mon8'), ('Gu', 'Tue9'),
('Gu', 'Wed10'), ('Gu', 'Thu11'), ('Gu', 'Fri12'), ('Gu', 'Sat13'),
('Gu', 'Sun14')
])

## Decision variables
The variable `x[w,s] = 1` if worker w is assigned to shift s; x 变量 表示是否排班
- Only use variables for _possible_ combinations - not when workers are unavailable
- Reduces model size and makes code efficient

In [4]:
# Create variables indexed by availability list
x = m.addVars(availability, ub=1, name="x")

## Objective
Minimize total salary cost

最小化薪水代价

In [5]:
m.setObjective(sum(pay[w]*x[w,s] for w,s in availability), GRB.MINIMIZE)

## Shift requirement constraints
For every shift, the number of workers must equal the shift requirement

约束条件: 每个班的工人需要满足排班量

In [6]:
reqCts = m.addConstrs((x.sum('*', s) == shiftRequirements[s] for s in shifts), "_")

与上面语句同样效果的写法

To understand the syntax, here is a less efficient version:

    for s in shifts:
        m.addConstr(sum(x[w,s] for w in workers
                               if (w,s) in availability)
                    == shiftRequirements[s])

The accompanying presentation explains the efficiency issues

## Solve
Remember that this model is designed to be infeasible

这是一个不可行的模型 (无法有效排班), 输出结果为 Infeasible model 不可行

In [7]:
m.optimize()

Optimize a model with 14 rows, 72 columns and 72 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [8e+00, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 7e+00]
Presolve removed 1 rows and 34 columns
Presolve time: 0.02s

Solved in 0 iterations and 0.02 seconds
Infeasible model


## Relax constraints
Call feasRelax to relax constraints

调用 feasRelax 放松变量进行求解

In [8]:
orignumvars = m.NumVars
m.feasRelaxS(0, False, False, True)
m.optimize()

m.printAttr('X', "Art*") # slack variables

Optimize a model with 14 rows, 100 columns and 100 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 7e+00]
Presolve removed 5 rows and 73 columns
Presolve time: 0.02s
Presolved: 9 rows, 27 columns, 27 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    6.0000000e+00   0.000000e+00   0.000000e+00      0s
       0    6.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.04 seconds
Optimal objective  6.000000000e+00

    Variable            X 
-------------------------
   ArtP_Thu4            2 
   ArtP_Sat6            2 
   ArtP_Sun7            1 
  ArtP_Fri12            1 


## Plot result
Show a heatmap of the shifts that cannot be satisfied

将上面的结果绘制成热力图; 可以看出 ArtP_Thu4 ArtP_Sat6 ArtP_Sun7 这几周的颜色更深( data['slacks']==0 浅蓝  == 1 中蓝 or == 2 深蓝 ), 没有满足;派克要求

In [9]:
from bokeh.charts import *

# output_notebook() # 嵌入式绘图

slackvars = m.getVars()[orignumvars:]
data = {
    'days': [s[:3] for s in shifts],
    'weeks': [1+i/7 for i in range(len(shifts))],
    'slacks': [v.X for v in slackvars if v.VarName.startswith("ArtP")],
}

show(HeatMap(data, x='days', y='weeks', values='slacks', stat=None, title="Shortfall"))

print data

{'weeks': [1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2], 'days': ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun', 'Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun'], 'slacks': [0.0, 0.0, 0.0, 2.0, 0.0, 2.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0]}
